# 01 · Exploración de Diccionarios, Datasets y Llaves

Este notebook **escanea y perfila** los archivos colocados en `data/external/` para:
- Listar **datasets** (.xlsx, .xls, .csv, .parquet) por entidad: `obra/`, `empresa/`, `funcionario/`, `catalogos/`.
- Detectar **hojas** en Excel y previsualizar **cabeceras**.
- Inferir **esquemas** (tipos, nulos, conteo único) y **posibles llaves**.
- Exportar inventarios a CSV para documentación.

> Objetivo: Dejar definido **qué archivo(s)** usaremos como base para construir el *dataset maestro* de **Obra** y cuáles serán las **claves de unión** con Empresa y Funcionario.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
import itertools

BASE = Path('..').resolve().parent / 'Deteccion_Corrupcion'
DATA_EXT = BASE / 'data' / 'external'
DATA_PROC = BASE / 'data' / 'processed'
DATA_EXT, DATA_PROC

(WindowsPath('C:/MaestriaUNI/Cursos/III-CICLO/TesisI/Solucion/Deteccion_Corrupcion/data/external'),
 WindowsPath('C:/MaestriaUNI/Cursos/III-CICLO/TesisI/Solucion/Deteccion_Corrupcion/data/processed'))

## 1) Escaneo de archivos por entidad
Se buscarán extensiones: `.xlsx`, `.xls`, `.csv`, `.parquet` en carpetas:
- `obra/`, `empresa/`, `funcionario/`, `catalogos/`

In [2]:
from typing import List, Dict

def scan_files(root: Path, sub: str) -> pd.DataFrame:
    folder = (root / sub)
    exts = ['*.xlsx','*.xls','*.csv','*.parquet']
    rows = []
    for pat in exts:
        for p in folder.rglob(pat):
            rows.append({
                'entidad': sub,
                'archivo': str(p.relative_to(root)),
                'nombre': p.name,
                'extension': p.suffix.lower(),
                'tamano_kb': round(p.stat().st_size/1024,2)
            })
    return pd.DataFrame(rows)

dfs = [scan_files(DATA_EXT, sub) for sub in ['obra','empresa','funcionario','catalogos']]
inv = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
inv.sort_values(['entidad','extension','archivo']).reset_index(drop=True)

,entidad,archivo,nombre,extension,tamano_kb
0,catalogos,catalogos\Diccionario vistas dashboards y data...,Diccionario vistas dashboards y datasets.xlsx,.xlsx,17.20
1,catalogos,catalogos\Diccionario_Datos_ML_Completo_V1.xlsx,Diccionario_Datos_ML_Completo_V1.xlsx,.xlsx,31.01
2,catalogos,catalogos\Diccionario_Datos_Sistemas_Fuente_V1...,Diccionario_Datos_Sistemas_Fuente_V1.xlsx,.xlsx,132.30
3,empresa,empresa\Datos generales - Empresa -2023.05.15....,Datos generales - Empresa -2023.05.15.xlsx,.xlsx,20.95
4,empresa,empresa\Perfilamiento de Empresa riesgosa_ CAT...,Perfilamiento de Empresa riesgosa_ CATEGORÍA X...,.xlsx,27.08
5,funcionario,funcionario\Datos generales - Funcionario - 20...,Datos generales - Funcionario - 2023.05.15.xlsx,.xlsx,21.63
6,funcionario,funcionario\Perfilamiento de Funcionario riesg...,Perfilamiento de Funcionario riesgoso_ CATEGOR...,.xlsx,27.89
7,obra,obra\Datos generales - Obra - 2023.05.15.xlsx,Datos generales - Obra - 2023.05.15.xlsx,.xlsx,21.74
8,obra,obra\Perfilamiento Obra Riesgosa_CATEGORÍA X_2...,Perfilamiento Obra Riesgosa_CATEGORÍA X_2023.0...,.xlsx,37.85


## 2) Guardar inventario de archivos
Se exporta un CSV con el listado completo para control documental.

In [3]:
out_inv = DATA_EXT / 'inventario_external_datasets.csv'
inv.to_csv(out_inv, index=False, encoding='utf-8')
out_inv, inv.shape

(WindowsPath('C:/MaestriaUNI/Cursos/III-CICLO/TesisI/Solucion/Deteccion_Corrupcion/data/external/inventario_external_datasets.csv'),
 (9, 5))

## 3) Utilitarios de lectura segura
Lectura de Excel/CSV/Parquet y utilitarios para previsualizar esquemas y llaves candidatas.

In [4]:
def leer_tabla(path: Path, sheet_name=None, nrows=None) -> pd.DataFrame:
    if not path.exists():
        print(f'[WARN] No existe: {path}')
        return pd.DataFrame()
    if path.suffix.lower() == '.csv':
        return pd.read_csv(path, nrows=nrows)
    if path.suffix.lower() in ('.xlsx','.xls'):
        return pd.read_excel(path, sheet_name=sheet_name, nrows=nrows)
    if path.suffix.lower() == '.parquet':
        return pd.read_parquet(path)
    print(f'[WARN] Extensión no soportada: {path.suffix}')
    return pd.DataFrame()

OBRA_KEYS = ['id_obra','id_snip','codigo_unico','id_seace','id_proyecto']
RUC_KEYS  = ['ruc_empresa','ruc','ruc_proveedor']
DNI_KEYS  = ['dni_funcionario','dni','id_miembro','doc_identidad']

def detectar_llaves(df: pd.DataFrame) -> Dict[str, list]:
    keys = {'obra':[], 'ruc':[], 'dni':[]}
    cols = [c.lower() for c in df.columns]
    for k in OBRA_KEYS:
        if k in cols: keys['obra'].append(k)
    for k in RUC_KEYS:
        if k in cols: keys['ruc'].append(k)
    for k in DNI_KEYS:
        if k in cols: keys['dni'].append(k)
    return keys

def perfil_basico(df: pd.DataFrame, max_cols=20) -> pd.DataFrame:
    prof = pd.DataFrame({
        'dtype': df.dtypes.astype(str),
        'n_nulos': df.isna().sum(),
        'pct_nulos': df.isna().mean().round(4),
        'n_unicos': df.nunique(dropna=False)
    })
    return prof.sort_values(['pct_nulos','n_unicos'], ascending=[False, True]).head(max_cols)

## 4) Exploración rápida por entidad
Para cada entidad, se mostrará un resumen por archivo y, en Excel, las hojas detectadas.

In [5]:
def listar_hojas_excel(path: Path) -> list:
    try:
        xls = pd.ExcelFile(path)
        return xls.sheet_names
    except Exception as e:
        print('[WARN] No se pudo leer hojas:', path.name, e)
        return []

resumen = []
for _, row in inv.iterrows():
    p = DATA_EXT / row['archivo']
    sheets = []
    if row['extension'] in ('.xlsx','.xls'):
        sheets = listar_hojas_excel(p)
    resumen.append({
        'entidad': row['entidad'],
        'archivo': row['archivo'],
        'extension': row['extension'],
        'tamano_kb': row['tamano_kb'],
        'sheets': ', '.join(sheets[:10]) if sheets else ''
    })
df_resumen = pd.DataFrame(resumen).sort_values(['entidad','extension','archivo']).reset_index(drop=True)
df_resumen.head(30)

,entidad,archivo,extension,tamano_kb,sheets
0,catalogos,catalogos\Diccionario vistas dashboards y data...,.xlsx,17.20,DICCIONARIO
1,catalogos,catalogos\Diccionario_Datos_ML_Completo_V1.xlsx,.xlsx,31.01,"1. MATRIZ_ML_DW, 2. OBRA_ML_DW, 3. EMPRESA_ML_..."
2,catalogos,catalogos\Diccionario_Datos_Sistemas_Fuente_V1...,.xlsx,132.30,"1. INVIERTE, 2. SEACE, 3. SIAF, 4. RNP, 5. SER..."
3,empresa,empresa\Datos generales - Empresa -2023.05.15....,.xlsx,20.95,Datos generales
4,empresa,empresa\Perfilamiento de Empresa riesgosa_ CAT...,.xlsx,27.08,"Indicadores, Datos generales"
5,funcionario,funcionario\Datos generales - Funcionario - 20...,.xlsx,21.63,Datos generales
6,funcionario,funcionario\Perfilamiento de Funcionario riesg...,.xlsx,27.89,"Indicadores y variables, Datos generales"
7,obra,obra\Datos generales - Obra - 2023.05.15.xlsx,.xlsx,21.74,Datos generales
8,obra,obra\Perfilamiento Obra Riesgosa_CATEGORÍA X_2...,.xlsx,37.85,"Indicadores Finales, Datos generales"


## 5) Guardar resumen y elegir muestras para *preview*
Se exporta `resumen_external_datasets.csv`. Abajo se puede configurar una lista corta de archivos a *previsualizar* (cabecera + perfil).

In [6]:
out_resumen = DATA_EXT / 'resumen_external_datasets.csv'
df_resumen.to_csv(out_resumen, index=False, encoding='utf-8')
out_resumen, df_resumen.shape

(WindowsPath('C:/MaestriaUNI/Cursos/III-CICLO/TesisI/Solucion/Deteccion_Corrupcion/data/external/resumen_external_datasets.csv'),
 (9, 5))

### Archivos a previsualizar (ajusta esta lista)

In [8]:
preview_files = [
    'obra/Perfilamiento Obra Riesgosa_CATEGORÍA X_2025.05.12.xlsx',
    'empresa/Perfilamiento de Empresa riesgosa_ CATEGORÍA X _2025.05.12.xlsx',
    'funcionario/Perfilamiento de Funcionario riesgoso_ CATEGORÍA X _2025.05.12.xlsx',
]
preview_files

['obra/Perfilamiento Obra Riesgosa_CATEGORÍA X_2025.05.12.xlsx',
 'empresa/Perfilamiento de Empresa riesgosa_ CATEGORÍA X _2025.05.12.xlsx',
 'funcionario/Perfilamiento de Funcionario riesgoso_ CATEGORÍA X _2025.05.12.xlsx']

## 6) Preview: cabeceras, perfiles y llaves candidatas
Para los archivos listados en `preview_files`, se hace una lectura parcial y se muestran perfiles básicos y llaves probables.

In [9]:
perfiles_export = []
for rel in preview_files:
    p = DATA_EXT / rel
    if not p.exists():
        print('[WARN] No existe para preview:', rel)
        continue
    try:
        sheet = 0 if p.suffix.lower() in ('.xlsx','.xls') else None
        df = leer_tabla(p, sheet_name=sheet, nrows=2000)
        if df.empty:
            print('[WARN] Vacío o no legible:', rel)
            continue
        display(df.head(5))
        prof = perfil_basico(df, max_cols=30)
        display(prof)
        keys = detectar_llaves(df)
        print('Llaves candidatas ->', keys)
        tmp = prof.reset_index().rename(columns={'index':'columna'})
        tmp.insert(0, 'archivo', rel)
        perfiles_export.append(tmp)
    except Exception as e:
        print('[ERROR] Preview falló en', rel, e)

if perfiles_export:
    df_perf = pd.concat(perfiles_export, ignore_index=True)
    out_perf = DATA_EXT / 'perfiles_preview.csv'
    df_perf.to_csv(out_perf, index=False, encoding='utf-8')
    out_perf

,N°,Código Interno,Categoría,Etapa,Fase (interno),Variable o Indicador,Fuente de información de donde provino la variable o indicador,Tipo,Base de datos,Tabla,Campos,Fórmula,Regla,Comentario,Notas extras,Norma que aplica
0,1,VOb1,Variable,Ejecución contractual,Ejecución de obra,Dias de ampliación de plazo acumulado,Diccionario datos SEACE - EJECUCIÓN,Riesgo,SEACE EJECUCION CONTRACTUAL,TBL_CON_AMPLIACION,N_PLAZO,Sumar los días de ampliación de plazo.,Ninguna,Ninguno,Ninguno,No aplica
1,2,VOb2,Variable,Ejecución contractual,Actividades iniciales /Ejecución de obras,Existencia de nulidad de contrato,Diccionario datos SEACE - EJECUCIÓN,Riesgo,SEACE EJECUCION CONTRACTUAL,TBL_CON_NULIDAD,N_ID_CAUSAL,Ninguna,Ninguna,Realizar búsqueda dentro de N_ID_CAUSAL el cau...,Ninguno,No aplica
2,3,VOb3,Variable,Ejecución contractual / Perfilamiento de Empresa,Actividades inciales,Número de contratos de obra suscritos en la mi...,Informe de consultores,Riesgo,SEACE SELECCIÓN,NO APLICA\n,Número de contrato\nFecha inicio (contrato)\nF...,Sumar los contratos de obra suscritos en la mi...,Ninguna,Ninguno,Ninguno,No aplica
3,4,VOb5,Variable,Ejecución contractual,Ejecución de obra,Monto contratado,Variables de Mx Obras,Importancia relativa,SEACE SELECCIÓN,NO APLICA,Monto (contratado),Ninguna,Ninguna,"Se considera, sin embargo queda a discriminaci...",Ninguno,No aplica
4,5,VOb6,Variable,Todas las etapas,Todas la etapas,Etapa actual de la obra pública (Actuaciones p...,Variables de Mx Obras,Importancia relativa,\nSEACE SELECCIÓN\nSEACE EJECUCIÓN CONTRACTUAL,NO APLICA,NO APLICA,Ninguna,Alto: Procedimiento de selección\nMedio: Ejecu...,Teniendo en cuenta la tabla de equivalencia se...,Se tiene que construir una tabla intermedia de...,No aplica


,dtype,n_nulos,pct_nulos,n_unicos
Notas extras,object,1,0.0238,7
Comentario,object,1,0.0238,24
Categoría,object,0,0.0000,2
Etapa,object,0,0.0000,6
Fuente de información de donde provino la variable o indicador,object,0,0.0000,7
Tipo,object,0,0.0000,7
Norma que aplica,object,0,0.0000,12
Fase (interno),object,0,0.0000,13
Base de datos,object,0,0.0000,19
Tabla,object,0,0.0000,22


Llaves candidatas -> {'obra': [], 'ruc': [], 'dni': []}


,N°,Código Interno,Categoría,Variable o Indicador,Fuente de información de donde provino la variable o indicador,Base de datos,Tabla,Campos,Fórmula,Regla,Comentario
0,1,I1E,Indicador,"¿El RUC de la empresa está activo, durante el ...",Diccionario de datos - SUNAT,SUNAT\nDB SEACE V3 - Selección,001\nNo aplica,NUMRUC\nNOMBRE\nESTADO\nFecha Registro de Part...,"ESTADO =""ACTIVO"" <> Fecha registro and Fecha f...","Alerta: ESTADO =""INACTIVO"" <> Fecha registro a...","En el campo ""ESTADO"" la empresa debe estar reg..."
1,2,I2E,Indicador,¿La empresa cuenta con RNP de ejecutor de obra...,Reunión de equipo interno,RNP,Base01a_borrador,LT_RUC\nRAZSOCIAL\nTIPO,TIPO = 1 (Ejecutor),Alerta: TIPO =! 1 (Ejecutor)\nSituación Normal...,Esta información debe ser consultada primero e...
2,3,I3E,Indicador,Monto de la capacidad libre de contratación,Informe de consultores,RNP\nBD SEACE V3 - Selección,Base01_borrador \nNo aplica,CAP_CONT\nFecha sucripción contrato\nMonto (Co...,i= Capacidad máxima de contratación sin consid...,Alerta: i <= 0\nSituación Normal: i > 0,"Para poder obtener el i, debo ubicarme en la f..."
3,4,I4E,Indicador,Empresas individuales extranjeras y empresas e...,Reunión de equipo interno,SUNAT,001\n002,UBIGEO\nNACIONALIDAD,Ninguno,Ninguna,Realizar el cruce entre la nacional del contri...
4,5,I5E,Indicador,Consorcios y/ o empresas que se hayan acogido ...,Reunión de equipo interno,BD SEACE V3 - Selección\nSUNAT,No aplica (Listado de expresión de interés)\n001,RUC/Código\nLey promoción de la Selva\nNUMRUC\...,Ninguno,RUC (Seace - Selección) = RUC (Sunat) <> Ley p...,Se debe realizar el cruce entre el RUC de la b...


,dtype,n_nulos,pct_nulos,n_unicos
Categoría,object,0,0.0,1
Fuente de información de donde provino la variable o indicador,object,0,0.0,4
Fórmula,object,0,0.0,7
Base de datos,object,0,0.0,8
Comentario,object,0,0.0,8
Tabla,object,0,0.0,9
Regla,object,0,0.0,9
N°,int64,0,0.0,10
Código Interno,object,0,0.0,10
Variable o Indicador,object,0,0.0,10


Llaves candidatas -> {'obra': [], 'ruc': [], 'dni': []}


,N°,Código Interno,Categoría,Variable o Indicador,Fuente de información de donde provino la variable o indicador,Base de datos,Tabla,Campos,Fórmula,Regla,Comentario,Clasificación de uso
0,1,V1F,Variable,Sexo del funcionario o servidor,Diccionario de datos - Airshp-MEF,SUNAT,002,SEXO,Ninguno,Ninguno,Ninguno,Datos generales - Reporte / Perfilamiento del ...
1,2,V2F,Variable,Edad del funcionario o servidor,Diccionario de datos - Airshp-MEF,SUNAT,002,FECNAC,Se debe realizar una resta entre la fecha de n...,Ninguno,Ninguno,Datos generales - Reporte / Perfilamiento del ...
2,3,V3F,Variable,Tipo de gobierno de la Entidad donde labora,Diccionario de datos - Airshp-MEF,Airshp-MEF,Tabla Activos\nTabla Cese,CODIGO_NIVEL\nDESC_NIVEL,Ninguno,Ninguno,Ninguno,Datos generales - Reporte
3,4,V4F,Variable,Sector de la Entidad donde labora,Diccionario de datos - Airshp-MEF,Airshp-MEF,Tabla Activos\nTabla Cese,CODIGO_SECTOR\nSECTOR,Ninguno,Ninguno,Ninguno,Datos generales - Reporte
4,5,V5F,Variable,Régimen laboral,Diccionario de datos - Airshp-MEF,Airshp-MEF,Tabla Activos\nTabla Cese,REGIMEN_LABORAL\nDESC_REGIMEN_LABORAL,Ninguno,Ninguno,Ninguno,Datos generales - Reporte


,dtype,n_nulos,pct_nulos,n_unicos
Categoría,object,0,0.0,2
Fuente de información de donde provino la variable o indicador,object,0,0.0,4
Clasificación de uso,object,0,0.0,4
Regla,object,0,0.0,5
Comentario,object,0,0.0,5
Base de datos,object,0,0.0,9
Tabla,object,0,0.0,10
Fórmula,object,0,0.0,10
Campos,object,0,0.0,16
N°,int64,0,0.0,18


Llaves candidatas -> {'obra': [], 'ruc': [], 'dni': []}
